## Set the repository path

In [ ]:
pyCPT_path = '../'

## Import modules

In [ ]:
# Add the repository path into python path
import sys
sys.path.append(pyCPT_path)

# Import modules
import numpy as np
import pyHMRF
import pyCPT
import post_process
import pre_process

## Read the NGES dataset

In [ ]:
datapath = pyCPT_path + '/test_data/NGES_data.csv'
cpt = pyCPT.read_cpt_data(datapath)
print('Number of data points:  ' + str(cpt.phys_shp))

## Identify the optimal number of soil states

In [ ]:
# mod_sel : the model selection result, 
# mod_sel[0] = BIC list for different number of clusters
# mod_sel[1] = the optimal number of clusters
mod_sel = pre_process.model_selection(cpt.feat, 10, plot=True)

## Extract soil segments

In [ ]:
pyCPT.segmentation(cpt, n=200, n_labels=mod_sel[1], start_iter=100)

## Soil interpretation

In [ ]:
layer_info = pyCPT.detect_layers(cpt)

## Visualization

In [ ]:
post_process.plot_feature(cpt,xlabel=['Fr','Qt'], scatter=False) # show the CPT data
post_process.plot_image(cpt, cpt.label_map_est) # show the spatial pattern
pyCPT.mixture_plot_robertsonchart(cpt, aspect=0.8) # show the statistical pattern
pyCPT.plot_layers(cpt, layer_info) # show the soil layers